In [34]:
import pandas as pd
import numpy as np

import nltk
import spacy

from gensim.models import Phrases
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

import pyLDAvis.gensim_models
import pyLDAvis

import warnings
warnings.filterwarnings('ignore')

In [53]:
df=pd.read_csv("frame2.csv")

In [54]:
df['words'] = df['words'].str.replace("'", "")

In [80]:
# Only take English transcripts
df = df[df.language == 'en']
df.language.value_counts()

language
en    398
Name: count, dtype: int64

### Clean text (get tri- and bi-grams, lemmatize, and take only selected POS)

In [56]:
# Build bigram and trigram Phrases objects
bigram_phrases = Phrases(df.words, min_count=10)
trigram_phrases = Phrases(bigram_phrases[df.words], min_count=5)

# Create Phraser model objects for faster processing
bigram_model = Phraser(bigram_phrases)
trigram_model = Phraser(trigram_phrases)
trigrams = [trigram_model[bigram_model[word.split()]] for word in df.words]

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
lemmatized_words = []
for sent in trigrams:
    doc = nlp(" ".join(sent))
    lemmatized_words.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [58]:
# Print the first 50 lemmatized words of the first row
print(lemmatized_words[0][:50])

['sledgehammer', 'call', 'name', 'thank', 'much', 'thank', 'thank', 'take', 'seat', 'thank', 'much', 'thank', 'come', 'man', 'let', 'ask', 'guy', 'man', 'go', 'keep', 'taking', 'confederate', 'statue', 'taking', 'temperature', 'room', 'know', 's', 'know', 'weird', 'start', 've', 'phenomenal', 'tour', 've', 've', 'meet', 'people', 'never', 'thought', 'meet', 'life', 'recently', 'meet', 'wild', 'think', 'want', 'meet', 'clear', 'fancy']


In [59]:
id2word = Dictionary(lemmatized_words)
id2word.filter_extremes(no_below=10, no_above=0.4)
id2word.compactify()
corpus = [id2word.doc2bow(word) for word in lemmatized_words]

### Create LDA model and print out topics

In [128]:
num_topics = 7
lda_model = LdaMulticore(corpus=corpus, 
                             id2word=id2word, 
                             num_topics=num_topics, 
                             random_state=1,
                             chunksize=30,
                             passes=40,
                             alpha=0.5,
                             eta=0.91,
                             eval_every=1,
                             per_word_topics=True,
                             workers=2)

In [129]:
lda_model.print_topics(7,num_words=10)

[(0,
  '0.005*"folk" + 0.005*"fart" + 0.004*"hat" + 0.004*"planet" + 0.003*"sport" + 0.003*"list" + 0.003*"television" + 0.003*"tit" + 0.003*"religion" + 0.003*"begin"'),
 (1,
  '0.008*"mate" + 0.007*"quite" + 0.005*"mum" + 0.005*"cheer" + 0.005*"lovely" + 0.004*"round" + 0.004*"film" + 0.004*"cock" + 0.004*"applause" + 0.004*"bloke"'),
 (2,
  '0.005*"police" + 0.005*"jail" + 0.005*"daddy" + 0.004*"folk" + 0.004*"weed" + 0.004*"president" + 0.004*"chicken" + 0.003*"btch" + 0.003*"goin" + 0.003*"rich"'),
 (3,
  '0.008*"fluffy" + 0.005*"rape" + 0.005*"freakin" + 0.004*"basket" + 0.004*"cake" + 0.004*"gift" + 0.004*"frankie" + 0.004*"snoop" + 0.004*"band" + 0.004*"freaking"'),
 (4,
  '0.012*"indian" + 0.007*"immigrant" + 0.007*"asian" + 0.006*"accent" + 0.006*"british" + 0.005*"racism" + 0.005*"german" + 0.005*"bro" + 0.005*"trevor" + 0.005*"snake"'),
 (5,
  '0.002*"awesome" + 0.002*"horrible" + 0.002*"husband" + 0.002*"cute" + 0.002*"ice" + 0.002*"coffee" + 0.002*"adult" + 0.002*"text" +

By looking at some of the key words we try to derive a topic:
- Topic 0 : "Culture"
- Topic 1 : "UK"
- Topic 2 : "Crimes"
- Topic 3 : "Situational"
- Topic 4 : "Immigrants"
- Topic 5 : "Relationships"
- Topic 6 : "Politics"

### Coherence Score

In [130]:
coherence_model_lda = CoherenceModel(model=lda_model, 
                                     texts=lemmatized_words, 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_model_lda.get_coherence()

0.34477313969112583

### The pyLDAvis

In [132]:
# Convert LDA model to a format compatible with pyLDAvis
pyLDAvis.enable_notebook()
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.026523  0.009823       1        1  47.321971
1     -0.012730  0.068500       2        1  14.671009
2      0.106719 -0.020737       3        1  12.982090
6     -0.084490 -0.026853       4        1  10.935677
0     -0.009920  0.082144       5        1   8.128526
4     -0.037610 -0.054351       6        1   3.611027
3      0.011509 -0.058526       7        1   2.349699, topic_info=           Term        Freq       Total Category  logprob  loglift
739   president  662.000000  662.000000  Default  30.0000  30.0000
1230      trump  441.000000  441.000000  Default  29.0000  29.0000
666        mate  377.000000  377.000000  Default  28.0000  28.0000
381       asian  308.000000  308.000000  Default  27.0000  27.0000
2920       rape  368.000000  368.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1944     garden   13.458024  120.006750   Topic7  -6.3184   1.5629
1302        bus   17.222475  319.411966   Topic7  -6.0717   0.8306
324      ticket   16.031050  317.555215   Topic7  -6.1434   0.7648
633        king   15.199687  272.765288   Topic7  -6.1967   0.8635
229      police   14.826391  425.850588   Topic7  -6.2215   0.3932

[448 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1825      1  0.549846  abortion
1825      2  0.019994  abortion
1825      3  0.029992  abortion
1825      4  0.164954  abortion
1825      5  0.219938  abortion
...     ...       ...       ...
356       3  0.058929      yell
356       4  0.046301      yell
356       5  0.008418      yell
356       6  0.008418      yell
356       7  0.012628      yell

[2709 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 3, 7, 1, 5, 4])

In [133]:
# Get a list of vectors of topic probabilities
topic_vecs = []
for i in range(len(df.words)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(num_topics)]
    topic_vecs.append(topic_vec)

In [134]:
# Add topic probabilities into main df. Create a new column for each topic.
topic_columns = ['Culture', 'UK', 'Crimes', 'Situational', 'Immigrants', 'Relationships', 'Politics']
LDA_probs = pd.DataFrame(data=topic_vecs, columns=topic_columns, index=df.index)
df = pd.concat([df, LDA_probs], axis=1)

In [136]:
# Saving the csv file
df.to_csv("frame3.csv", index=False)